# Tramento dos dados brutos

Após a carga dos dataset a partir dos arquivos (doCarga), nesta etapa os dados sofrerão limpesa e tratamento, aonde serão retiradas "sugeiras grosseiras"

In [1]:
import nltk
from nltk.corpus import stopwords
import unicodedata
import pandas as pd

# Tratamentos de Colunas

In [2]:
# retira stop words do nome dos usuários reais
def limpaTexto( entrada ):
    
    listaRetiraInicio = [] # ['BB','Z ', 'MC ']
    listaRetiraFim    = [] # [' CC']
    listaCaracteresIndesejados = ["	", # tabulação
                                "*",
                                "#",
                                "|",
                                '¥'
                                ]

    # PROBLEMA: TEM QUE VERIFICAR A LISTA DE STOPWORDS PARA QUE ELA 
    # NÃO RETIRE DESCRIÇÃO VÁLIDA DE PRODUTOS.
    # retira stopwords
    entrada = str(entrada)
    entrada = entrada.split(" ")
    saida = ""
    for item in entrada:
        if item.lower() not in lstStopWords:
            saida = saida + item + " "
    
    # retira acentos
    aux   = str(saida)
    nfkd  = unicodedata.normalize('NFKD', aux)
    saida = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    
    # transforma em maiúsculo, retira espaços dos extremos (TRIM), 
    saida = saida.upper().strip()
    
    # retira caracteres indesejados --> cuidado para não unir palavras separadas pelo caracter
    for item in listaCaracteresIndesejados:
        saida = saida.replace(item,"")    
    
    # retira espaços duplos dentro da string
    saida = saida.replace("  "," ")
    while saida.find('  ') != -1:
        saida = saida.replace("  "," ")
              
    # retira inicios de strings que constam na lista listaRetiraInicio
    for item in listaRetiraInicio:
        # retira espaços duplos dentro do item, pois o mesmo já foi feito no texto a pesquisar
        item = item.replace("  "," ")
        while item.find('  ') != -1:
            item = item.replace("  "," ")
            
        if saida.startswith(item):
            tamanhoItem  = len(item)
            tamanhoSaida = len(saida)            
            if tamanhoSaida > tamanhoItem:
                saida = saida[tamanhoItem:]
    
    # retira fins de strings que constam na lista listaRetiraFim
    for item in listaRetiraFim:
        # retira espaços duplos dentro do item, pois o mesmo já foi feito no texto a pesquisar
        item = item.replace("  "," ")
        while item.find('  ') != -1:
            item = item.replace("  "," ")        
        
        if saida.endswith(item):
            tamanhoItem  = len(item)
            tamanhoSaida = len(saida)
            if tamanhoSaida > tamanhoItem:
                saida = saida[:tamanhoSaida-tamanhoItem]   

    # retira espaços duplos dentro da string. Necessário pois pode ter sobrado 
    # espaço depois das alterações
    saida = saida.strip()
        
    return saida 

In [3]:
def doLimpeza(df_original):
    # Apaga linhas cuja coluna 'NovaDescricao' esteja em branco
    df_original['NovaDescricao'].dropna( axis=0, inplace=True ) 

    # Apaga colunas que não serão usadas
    df_original.drop(['Desc_Catalogo', 'NCM_NFe', 'Desc_Anexo_IV', 'NCM_Calc_Prov', 'Cor_NCM_Calc', 'NCM_Calc', 
         'Item_Anexo_IV', 'Prod_ST', 'Prod_FCP'], axis=1, inplace=True)
    
    return df_original

In [4]:
def doDropDuplicados(df_original):
    ## Elimina linhas inconsistentes, repetidas. Mantém a primeira delas
    antes = df_original.shape[0]
    df_original.drop_duplicates( subset = ["Categoria"], inplace = True) 
    print( 'Eliminados', antes - df_original.shape[0], 'registros')
    del antes
    return df_original 

In [5]:
def doDropIndesejados(df_original):
    # Retira categorias indesejadas
    retirar = ['nannãonão','nansimnão','nansimsim']
    mask = ~df_original['Categoria'].isin(retirar)
    filtrado = df_original[mask].copy()
    df_original = filtrado.copy()
    del filtrado, retirar
    return df_original

# Tratamento de Strings

In [6]:
def doListaPalavras(df):
    # Gera lista com todas as palavras, com repetições
    # CountVectorizer retira duplicidade
    total_linhas = df.shape[0]
    lista_palavras = []
    for i in range(0, total_linhas):
        texto = df.iloc[i, 2].split()
        tamanho_linha = len(texto)
        for j in range(0,tamanho_linha):
            lista_palavras.append(texto[j]) 
            
    # Gera DF com a contagem de cada palavra
    lista_palavras = pd.Series(data = lista_palavras)
    df_lista_palavras = pd.DataFrame(data = lista_palavras.value_counts() )#, index = lista_palavras)
    df_lista_palavras.reset_index(drop = False, inplace = True)
    df_lista_palavras.columns = ['Palavra', 'QTD']

    #libera memoria
    del total_linhas, lista_palavras, texto, tamanho_linha,
    return df_lista_palavras

In [7]:
def doRemoveChar(Lista):
    # elimina palavras de 1 caracter
    mask = (Lista['Palavra'].str.len() == 1)
    df_aux = Lista[mask]
    Eliminar = []
    Eliminar = list(df_aux['Palavra'])
    mask = ~(Lista['Palavra'].str.len() == 1)
    df_aux = Lista[mask]
    Lista = df_aux.copy()

    del df_aux, mask
    return Lista, Eliminar

In [8]:
def doRemoveNumeros(Lista, Eliminar):
    # Elimina palavras que são somente numeros, exceto os de 13 algarismos ( possíveis EAN)

    # Separa possíveis códigos EAN
    mask1 = Lista.Palavra.str.len() == 13
    mask2 = Lista.Palavra.str.isdigit()
    df_aux = Lista[~(mask1 & mask2)]
    # elimina números
    mask2 = df_aux.Palavra.str.isdigit()
    df_aux = df_aux[mask2]

    Eliminar = Eliminar + list(df_aux['Palavra'])
    mask = ~Lista['Palavra'].isin( list(df_aux['Palavra']) )
    df_aux = Lista[mask]
    Lista = df_aux.copy()
    del df_aux, mask1, mask2
    return Lista, Eliminar

# Tratamento de Arquivos

In [9]:
def doMapOcorrencia(Origem, termo):
    result = False
    mask1  = Origem['Palavra'].str.endswith(termo)
    mask2  = Origem['Palavra'].str.len() < 80
    mask3  = Origem['QTD'] > 10 
    df_aux = Origem[ mask1 & mask2 & mask3 ]

    if df_aux.shape[0] > 1:
        df_aux.to_csv("ETL_" + termo + "_a_verificar.csv", index = True, columns = ["Palavra", "QTD"], sep = ';', encoding = "utf-8")
        result = True
    else:
        print( 'Não há palavras terminadas com "{}", ou há somente uma.'.format(termo) )
    del df_aux, mask1, mask2, mask3
    return result 

def doMapQtd(Origem):
    #Trata palavras pela quantidade delas
    result = False
    mask  = Origem['QTD'] > 1000 
    df_aux = Origem[ mask ]
    if df_aux.shape[0] > 1:
        df_aux.to_csv("ETL_QUANTIDADE_a_verificar.csv", index = True, columns = ["Palavra", "QTD"], sep = ';', encoding = "utf-8")
        result = True
    else:
        print( 'Não há palavras mais de 1000 ocorrências' )
    del df_aux, mask
    return result

In [10]:
def EliminaPalavrasVindasDeArquivo( nome_arquivo, Lista ):
    df_aux = pd.read_csv(nome_arquivo, index_col = ['Indice'], sep = ';',)
    Lista.append(list(df_aux['Palavra']))
    mask = ~Lista['Palavra'].isin( list(df_aux['Palavra']) )
    df_aux = Lista[mask]
    Lista = df_aux.copy()
    del df_aux, mask
    return Lista

# Execução

In [11]:
def doTrataColunas(Origem):
    ## TRATAMENTO DE COLUNAS ##

    # Concatena 3 colunas para formar a coluna 'Categoria', que é a classificação do produto
    Origem['Categoria'] = Origem['Item_Anexo_IV'].astype(str) + Origem['Prod_ST'] + Origem['Prod_FCP']
    Origem["NovaDescricao"] = limpaTexto(Origem['Desc_Catalogo'])
    Origem = doLimpeza(Origem)    

    # Renomeia as colunas do DF
    #df_original.columns = ('Remetente', 'NovaDescricao', 'Categoria') ## <<< ACHO Q ESTA INVERTIDO !!!!!!!!!!!!!!
    Origem.columns = ('Remetente', 'Categoria', 'NovaDescricao') 

    Origem = doDropDuplicados(Origem)
    Origem = doDropIndesejados(Origem)
    return Origem

In [12]:
def doTrataTextos(Origem):
    ## TRATAMENTO DE TEXTOS ##
    Palavras = doListaPalavras(Origem)          # Cria a lista Palavras
    Palavras, Eliminar = doRemoveChar(Palavras) # Cria a lista Eliminar
    Palavras, Eliminar = doRemoveNumeros(Palavras, Eliminar)
    return Origem, Palavras, Eliminar

In [13]:
def doTrataArquivos(Palavras, Eliminar, Caminho):
    ## TRATAMENTO DE ARQUIVOS ##
    # Gera arquivos
    if (doMapOcorrencia(Palavras, "ML")):
        Palavras = EliminaPalavrasVindasDeArquivo( Caminho + "ETL_ML_verificado.csv", Palavras )
    if (doMapOcorrencia(Palavras, "KG")):
        Palavras = EliminaPalavrasVindasDeArquivo( Caminho + "ETL_KG_verificado.csv", Palavras )
    if (doMapOcorrencia(Palavras, "GR")):
        Palavras = EliminaPalavrasVindasDeArquivo( Caminho + "ETL_GR_verificado.csv", Palavras )
    if (doMapOcorrencia(Palavras, "GRAMAS")):
        Palavras = EliminaPalavrasVindasDeArquivo( Caminho + "ETL_GRAMAS_verificado.csv", Palavras )
    if (doMapOcorrencia(Palavras, "GRAMA")):
        Palavras = EliminaPalavrasVindasDeArquivo( Caminho + "ETL_GRAMA_verificado.csv", Palavras )
    if (doMapOcorrencia(Palavras, "G")):
        Palavras = EliminaPalavrasVindasDeArquivo( Caminho + "ETL_G_verificado.csv", Palavras )
    if (doMapQtd(Palavras)):
        Palavras = EliminaPalavrasVindasDeArquivo( Caminho + "ETL_QUANTIDADE_verificado.csv", Palavras )

    return Palavras, Eliminar

In [14]:
def verificaDescricao(frase, Dicionario):
    resp = ''
   # palavraPermitida = df_lista_palavras['Palavra']
   # dicPermitido = dict(zip(palavraPermitida, palavraPermitida))

    frase = frase.split(" ")
    for palavra in frase:
        resp = resp + " " + Dicionario.get(palavra,'')
    resp = resp.strip()
    resp = resp.replace("  "," ")
    return resp

In [15]:
def doGravaPalavras(Origem, Caminho):
    #Salva as palavras da base, já com as devidas retiradas de palavras
    Origem.to_csv(Caminho + "ETL_base_lista_palavras_corretas.csv", index = True, columns = ["Palavra", "QTD"], sep = ';', encoding = "utf-8")
    Origem.to_pickle(Caminho + "ETL_base_lista_palavras_corretas.pkl")

def doGrava(Origem, Caminho):
    Origem.to_pickle(Caminho + "ETL_base_pronta_para_previsao.pkl")

In [16]:
def doTratamento(Original, Caminho = "..\\dados\\"):
    lstStopWords = set(stopwords.words('portuguese') )
    
    Original = doTrataColunas(Original)
    Eliminar = []    
    Original, Palavras, Eliminar = doTrataTextos(Original)
    
    Palavras, Eliminar = doTrataArquivos(Palavras, Eliminar, Caminho)
    
    doGravaPalavras(Palavras, Caminho)
    
    #aux = list( map(lambda x: verificaDescricao(x, Palavras), list(Original['NovaDescricao']) ) ) 
    #Original['NovaDescricao'] = aux
    
    print ('kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk')
    print (Palavras)
    print ('dddddddddddddddddddddddddddddddddddddddddddddddddddddd')

    doGrava(Palavras, Caminho)
    return Original

In [17]:
# SANDBOX

import import_ipynb
from carga import *
nltk.download('stopwords')
lstStopWords = set(stopwords.words('portuguese'))
diretorio = '..\\dados\\'
df_original = doCarga(diretorio, opcao=1)  # 2 df_originalAnonimo

doTratamento(df_original)

#df_original

importing Jupyter notebook from carga.ipynb


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\svpon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<string>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
